In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.3-70b-versatile")

model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001714AB35C10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001714AC7CE50>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

### RunnablePassthrough
- It does not do anything to the input data.
- Let's see it in a very simple example: a chain with just RunnablePassthrough() will output the original input without any modification.

In [3]:
from langchain_core.runnables import RunnablePassthrough

RPT = RunnablePassthrough()

RPT.invoke("ShwaTech")

'ShwaTech'

### RunnableLambda
- To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.
- Let's define a very simple function to create Russian lastnames:

In [4]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

In [5]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

chain.invoke("ShwaTech")

'ShwaTechovich'

### RunnableParallel
- We will use RunnableParallel() for running tasks in parallel.
- This is probably the most important and most useful Runnable from LangChain.
- In the following chain, RunnableParallel is going to run these two tasks in parallel :
    - operation_a will use RunnablePassthrough.
    - operation_b will use RunnableLambda with the russian_lastname function.

In [6]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operator_a": RunnablePassthrough(),
        "operator_b": RunnableLambda(russian_lastname)
    }
)

chain.invoke("ShwaCode")

{'operator_a': 'ShwaCode', 'operator_b': 'ShwaCodeovich'}

Example ???

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [8]:
prompt = ChatPromptTemplate.from_template("Explain me in simple terms what is {topic} in relation to Quantum Computing")

output_parser = StrOutputParser()

In [9]:
def simplefunc(topic):
    return topic["name"] + " Field"

In [10]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

runnables = RunnableParallel(
    {
        "operator_a": RunnablePassthrough(),
        "topic": RunnableLambda(simplefunc),
        "operator_c": RunnablePassthrough(),
    }
)

In [11]:
chain = runnables | prompt | model | output_parser

In [12]:
chain.invoke({
    "name": "Quantum Optics"
})

"I'd be happy to explain Quantum Optics in simple terms, especially in relation to Quantum Computing.\n\n**What is Quantum Optics?**\n\nQuantum Optics is a field of study that combines two areas: Quantum Mechanics and Optics. It explores how light (photons) behaves at a quantum level, which is the smallest scale where strange and fascinating phenomena occur.\n\n**Key aspects of Quantum Optics:**\n\n1. **Photons**: Quantum Optics studies the behavior of individual photons, which are the particles that make up light.\n2. **Quantum properties**: Photons can exhibit unique properties like superposition (being in multiple states at once), entanglement (being connected to each other), and interference (canceling or reinforcing each other).\n3. **Light-matter interactions**: Quantum Optics examines how photons interact with matter, such as atoms, molecules, or special materials.\n\n**Connection to Quantum Computing:**\n\nQuantum Optics plays a crucial role in Quantum Computing, as it provides

### Let's see a more advanced use of RunnableParallel

In [13]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_groq import ChatGroq
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [15]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vector_store = FAISS.from_texts(
    ["ShwaTech focuses on providing content on Data Science, AI, ML, DL, CV, NLP, LLM, Gen AI, Agentic AI, Quantum Computing, QML, QAI and Competitive Programming etc. in English."],
    embedding=embeddings
)

In [16]:
retriever = vector_store.as_retriever()

In [17]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

In [18]:
prompt = ChatPromptTemplate.from_template(template)

In [19]:
model = ChatGroq(model="llama-3.3-70b-versatile")

In [ ]:
output_parser = StrOutputParser()

In [20]:
retrieval_chain = (
    RunnableParallel({
        "context": retriever,
        "question": RunnablePassthrough(),
    })
    | prompt
    | model
    | output_parser
)

In [21]:
retrieval_chain.invoke("What is ShwaTech Organization?")

'ShwaTech is an organization that focuses on providing content related to various technological fields, including Data Science, Artificial Intelligence (AI), Machine Learning (ML), Deep Learning (DL), Computer Vision (CV), Natural Language Processing (NLP), Large Language Models (LLM), General AI, Agentic AI, Quantum Computing, Quantum Machine Learning (QML), Quantum AI, and Competitive Programming, with all content provided in English.'

In [22]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq
from langchain_google_genai import GoogleGenerativeAIEmbeddings

model = ChatGroq(model="llama-3.3-70b-versatile")

embeddings = GoogleGenerativeAIEmbeddings(model="text-embedding-004")

vectorstore = FAISS.from_texts(
    ["ShwaTech focuses on providing content on Data Science, AI, ML, DL, CV, NLP, LLM, Generative AI, Agentic AI, Quantum Computing, QML, QAI and Competitive Programming etc. in English."],
    embedding=embeddings,
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {  ## If Noticed I Donot Use RunnableParallel, As It is Used Behind the scene if we use {} 
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "What is ShwaTech Organization?", "language": "Pirate English"})

"ShwaTech be a swashbucklin' organization that sets its sights on providin' treasure troves o' content, matey! They be focusin' on Data Science, AI, ML, DL, CV, NLP, LLM, Generative AI, Agentic AI, Quantum Computing, QML, QAI, and Competitive Programming, savvy? And they be servin' up all this booty in the English language, arrr!"